# Exploration for Reported Assessment Results

## Imports and Such

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import geopandas as gpd
from shapely.geometry import Point
import plotly.graph_objects as go
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [ ]:
# Import assessment data minus fully suppressed scores
assessments = pd.read_pickle('../data/school_based/assessments_clean.pkl')

# Import Suppressed outlier data
suppressed = pd.read_pickle ('../data/school_based/full_suppression.pkl')

# Import Tennessee School District Geometry
tn_leas = gpd.read_file('../data/tn_leas.geojson', index_col='system_name')

## Listy McListface - A Place to look at the lists in my dataframe.

In [ ]:
# Assessments Info
assessments.info()

In [ ]:
# Student Groups
student_group_list = np.unique(assessments['student_group'].values).tolist()
student_group_list

In [ ]:
# School Types
school_type_list = np.unique(assessments['school_type'].values).tolist()
school_type_list

In [ ]:
# Subject Areas
subject_area_list = np.unique(assessments['subject_area'].values).tolist()
subject_area_list

## Unsuppressed: Broad Overview of Results

### Overall Unweighted Proficiencies by School-Level, Subject Area, and Year.

In [ ]:
# Subject Area Pivot Table
subject_area_pivot = pd.pivot_table(assessments,
                       values = 'pct_met_exceeded',
                       index = ['school_lvl'],
                       columns = ['subject_area','year'],
                       aggfunc = np.mean)

# Get the current list of years
years = list(subject_area_pivot.columns)

subject_area_pivot
#.plot()

#### Visual of Log Proficiencies For All Subjects Combined by School Level (Unweighted)

In [ ]:
# Apply logarithmic scaling to the values
log_values = np.log(subject_area_pivot.values)

# Create a diverging colorscale for heatmap
colorscale = 'Viridis'

# Create the heatmap figure
fig = go.Figure(data=go.Heatmap(
    z=log_values,
    x=subject_area_pivot.columns.get_level_values('year'),
    y=subject_area_pivot.index,
    colorscale=colorscale,
    zmid=np.median(log_values)  # Set the midpoint of the colorscale
))

# Update the layout
fig.update_layout(
    title='Subject Area Heatmap (Log Scale)',
    xaxis_title='Year',
    yaxis_title='School Level'
)

# Show the figure
fig.show()

### 🗺️ Spatial Join of School Geometry (lat/long point) and District (polygons)

There's some funk going on here.  District names don't match across datasets.  I'm going to do a spatial merge to see which dististricts are associated based on thier physical location.

In [ ]:
# Set the CRS for the assessments dataframe
assessments.crs = "EPSG:4269"

# Reproject assessments dataframe to match the CRS of tn_leas dataframe
reproject = assessments.to_crs(tn_leas.crs)

# Perform spatial join
assessments = gpd.sjoin(reproject, tn_leas, how='inner', predicate='intersects')

There is indeed a mismatch in naming conventions between datasets

In [ ]:
# Looking at differences in naming conventions
pd.merge(
    left = assessments.groupby(['system_name_left','school_name'])['system_name_right'].nunique().loc[lambda x: x>1].reset_index().drop(columns = 'system_name_right'),
    right = assessments)[['system_name_left', 'school_name', 'system_name_right']].drop_duplicates()#.to_csv('../data/fixerupper.csv', index = False)

subset_assessments = assessments[['system_name_left', 'system_name_right']]
subset_assessments.tail()

In [ ]:
# Load cleaned dictionary mapping
clean_dictionary =pd.read_csv('../data/clean_dictionary.csv')
clean_dictionary = pd.concat([pd.merge(
    left = assessments.groupby(['system_name_left','school_name'])['system_name_right'].nunique().loc[lambda x: x == 1].reset_index().drop(columns = 'system_name_right'),
    right = assessments)[['system_name_left', 'school_name', 'system_name_right']].drop_duplicates(), clean_dictionary])
clean_dictionary.tail(n=7)

Proccessing steps: Keep system_name_right, rename as system_name, drop system_name_right, set_system_name at col index 2

In [ ]:
# Merge assessments with the clean dictionary
assessments = (pd.merge(left = assessments, right = clean_dictionary))

# Rename the 'system_name_right' column to 'system_name'
assessments.rename(columns={'system_name_right': 'system_name'}, inplace=True)

# Move the 'system_name' column to the third position
columns = list(assessments.columns)
columns.insert(2, columns.pop(columns.index('system_name')))
assessments = assessments[columns]

# Drop the 'system_name_right' column
assessments.drop('system_name_left', axis=1, inplace=True)

In [ ]:
assessments.head()

#### 🏫 List of columns in assessments (school level) for use in district analysis.

In [ ]:
# Let's figure how how I can pivot this    
districts = assessments[['locale',
                         'year',
                         'system_name', 
                         'school_lvl', 
                         'subject_area', 
                         'student_group', 
                         'pct_met_exceeded_w', 
                         'school_type',
                         'magnet',
                         'charter',
                         'title_1',
                         'fte_teachers_w',
                         'stu_tchr_ratio_w',
                         'valid_tests']]

In [ ]:
unique_system_names = districts['system_name'].unique()
unique_system_names

### 🏋️ Weighting Metrics Based on Valid Tests

#### 📇 Indices for Weight Pivots

In [ ]:
# Indices for pivots
indices = ['system_name',  # School District 
           'school_lvl', # Level of school (Elem, Middle, High)
           'school_type', # Regular, alternative, special education
           'magnet', # Is magnet?
           'charter', # Is charter?
           'title_1', # Is title 1?
           'locale', # Location category of school (rural, large city, etc)
           'subject_area', # Overall content area of 
           'student_group'] # Aggregate student groups (all students, students with disabilities, etc)

#### 🏋️➕ Sum of Valid-Test-Weighted Scores for pct_met_exceeded, fte_teachers, and student_tchr_ratio 

In [ ]:
# Sum of valid test scores (The 🏋️)
weight = pd.pivot_table(
    districts,
    values='valid_tests',  # Column to calculate the sum of valid test scores
    index=indices,
    columns='year',
    aggfunc=np.sum
)

# Sum of weighted scores pivot for 'pct_met_exceeded'
sum_weighted_proficiency = pd.pivot_table(
    districts,
    values='pct_met_exceeded_w', # Weighted sum of students who met or exceeded expectations
    index=indices,
    columns='year',
    aggfunc=np.sum
)

# Sum of weighted scores pivot for 'fte_teachers'
sum_weighted_fte = pd.pivot_table(
    districts,
    values='fte_teachers_w', # Weighted sum of full-time equivalent teachers
    index=indices,
    columns='year',
    aggfunc=np.sum
)

# Sum of weighted scores pivot for 'stu_tchr_ratio'
sum_weighted_str = pd.pivot_table(
    districts,
    values='stu_tchr_ratio_w', # Weighted sum of student/teacher ratios
    index=indices,
    columns='year',
    aggfunc=np.sum
)

# Create a multi-level column index
column_index = pd.MultiIndex.from_product([['pct_met_exceeded', 'fte_teachers', 'stu_tchr_ratio'], sum_weighted_proficiency.columns])

# Concatenate the pivot tables horizontally
weighted_sums_pivot = pd.concat([sum_weighted_proficiency, sum_weighted_fte, sum_weighted_str], axis=1)
weighted_sums_pivot.columns = column_index

#### 🏋️⚖️ Weighted Averages for pct_met_exceeded_w, fte_teachers_w, stu_tchr_ratio_w

In [ ]:
# Divide sum_weighted_proficiency by weight
weighted_avg_proficiency = sum_weighted_proficiency / weight

# Divide sum_weighted_fte by weight
weighted_avg_fte = sum_weighted_fte / weight

# Divide sum_weighted_str by weight
weighted_avg_str = sum_weighted_str / weight

# Create a multi-level column index for the weighted average pivots
column_index = pd.MultiIndex.from_product([['pct_met_exceeded', 'fte_teachers', 'stu_tchr_ratio'], weighted_avg_proficiency.columns])

# Concatenate the weighted average pivots horizontally
weighted_avg_pivot = pd.concat([weighted_avg_proficiency, weighted_avg_fte, weighted_avg_str], axis=1)
weighted_avg_pivot.columns = column_index

weighted_avg_pivot

### 🤺 Slicing and Lagging
> "... the heavy-sword splendid.  The hard-edgèd weapon;  with Hrunting to aid me, I shall gain me glory .. "
    >> *-Beowulf*

#### 🪟 Setting Variables for lag window slicing
> 🗒️ **lvpp lags** → The lags for **Science** related metrics require some special treatment. The last valid prepandemic measurement for **ELA**, **Math**, and **Social Studies** was in **2019**.   **Science**, however, was not assessed in **2019**, but was in **2018**.  I cannnot directly compare lag windows for all subjects that start in **2019**.  In order to include metrics related to **Science** for fair comparisons, its window must start at **2018**.  Therefore, **lvpp** variables will be used as treatment for window the **last valid prepandemic measure** for each content area (e.g. 2018 or 2019 respectively).

> 🗂️ lvpp → last valid pre-pandemic assessment to 2021 (first full school year after 2020 school closure)

> 🗂️ intra → difference between first and second years school were reopend post-pandemic (2021 - 2022)

> 🗂️ pre_post → difference between last valid pre-pandemic scores and last year in the dataset (lvpp to 2022)

In [ ]:
# Set the stop year for all metrics
lvpp_stop = 2021

# Set the start year for Math, ELA, and Social Studies related metrics
lvpp_start_mess = 2019

# Set the start year for Science-related metrics
lvpp_start_science = 2018

# Set the start and stop year for the "intra" lag
intra_start = 2021
intra_stop = 2022

# Science Slicer
science_slice = (slice(None), slice(None), slice(None), slice(None), slice(None), slice(None), slice(None), 'Science')

# Non-Science slicer
subjects_slice = (slice(None), slice(None), slice(None), slice(None), slice(None), slice(None), slice(None))

#### 📝 Assessment Proficiency Lags

> 🗂️ pct_met_exceeded → Changes in weighted average full-time students who displayed **at-least minimum expected proficiency** over time.

In [ ]:
# Calculate separate lag scores for Science-related metrics.

# Last Valid Pre-Pandemic met_exceded measurement (lvpp)
weighted_avg_pivot.loc[science_slice, ('pct_met_exceeded', 'lvpp')] = (
    weighted_avg_pivot.loc[science_slice, ('pct_met_exceeded', lvpp_stop)] -
    weighted_avg_pivot.loc[science_slice, ('pct_met_exceeded', lvpp_start_science)]
)

# Calculate the lvpp scores for Math, ELA, and Social Studies related metrics
weighted_avg_pivot.loc[
    subjects_slice + (['Math', 'ELA', 'Social Studies'],),
    ('pct_met_exceeded', 'lvpp')
] = (
    weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('pct_met_exceeded', lvpp_stop)
    ] - weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('pct_met_exceeded', lvpp_start_mess)
    ]
)

# Calculate the intra lag score for 'pct_met_exceeded'
weighted_avg_pivot.loc[subjects_slice, ('pct_met_exceeded', 'intra')] = (
    weighted_avg_pivot.loc[subjects_slice, ('pct_met_exceeded', intra_stop)] -
    weighted_avg_pivot.loc[subjects_slice, ('pct_met_exceeded', intra_start)]
)

# Calculate the 'pre-post' assessment lag scores
weighted_avg_pivot[('pct_met_exceeded', 'pre-post')] = (
    weighted_avg_pivot[('pct_met_exceeded', 2022)] -
    weighted_avg_pivot[('pct_met_exceeded', 'lvpp')]
)

#### 🧑‍🏫 Full Time Equivalent Teachers Lag

> 🗂️ fte_teachers → Changes in weighted-average of **full-time-equivalent teachers** over time.

In [ ]:
# Last Valid Pre-Pandemic science-related full-time equivalent teachers measurement (lvpp)
weighted_avg_pivot.loc[science_slice, ('fte_teachers', 'lvpp')] = (
    weighted_avg_pivot.loc[science_slice, ('fte_teachers', lvpp_stop)] -
    weighted_avg_pivot.loc[science_slice, ('fte_teachers', lvpp_start_science)]
)

# Last Valid Pre-Pandemic full-time equivalent teacher measurement (lvpp)
weighted_avg_pivot.loc[
    subjects_slice + (['Math', 'ELA', 'Social Studies'],),
    ('fte_teachers', 'lvpp')
] = (
    weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('fte_teachers', lvpp_stop)
    ] - weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('fte_teachers', lvpp_start_mess)
    ]
)

# Calculate the intra lag score for 'fte_teachers'
weighted_avg_pivot.loc[subjects_slice, ('fte_teachers', 'intra')] = (
    weighted_avg_pivot.loc[subjects_slice, ('fte_teachers', intra_stop)] -
    weighted_avg_pivot.loc[subjects_slice, ('fte_teachers', intra_start)]
)

# Calculate the 'pre-post' assessment lag scores for full-time equivalent teachers
weighted_avg_pivot[('fte_teachers', 'pre-post')] = (
    weighted_avg_pivot[('fte_teachers', 2022)] -
    weighted_avg_pivot[('fte_teachers', 'lvpp')]
)

#### 🧑‍🎓/🧑‍🏫 Student Teacher Ratio Lag

> 🗂️ stu_tchr_ratio → Changes in weighted-average **student-to-teacher ratio** over time.

In [ ]:
# Last Valid Pre-Pandemic science-related student/teacher ratio measurement (lvpp)
weighted_avg_pivot.loc[science_slice, ('stu_tchr_ratio', 'lvpp')] = (
    weighted_avg_pivot.loc[science_slice, ('stu_tchr_ratio', lvpp_stop)] -
    weighted_avg_pivot.loc[science_slice, ('stu_tchr_ratio', lvpp_start_science)]
)

# Last Valid Pre-Pandemic non science-related student/teacher ratio measurement (lvpp)
weighted_avg_pivot.loc[
    subjects_slice + (['Math', 'ELA', 'Social Studies'],),
    ('stu_tchr_ratio', 'lvpp')
] = (
    weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('stu_tchr_ratio', lvpp_stop)
    ] - weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('stu_tchr_ratio', lvpp_start_mess)
    ]
)

# Calculate the intra lag score for 'stu_tchr_ratio'
weighted_avg_pivot.loc[subjects_slice, ('stu_tchr_ratio', 'intra')] = (
    weighted_avg_pivot.loc[subjects_slice, ('stu_tchr_ratio', intra_stop)] -
    weighted_avg_pivot.loc[subjects_slice, ('stu_tchr_ratio', intra_start)]
)

# Calculate the 'pre-post' assessment lag scores for student/teacher ratios
weighted_avg_pivot[('stu_tchr_ratio', 'pre-post')] = (
    weighted_avg_pivot[('stu_tchr_ratio', 2022)] -
    weighted_avg_pivot[('stu_tchr_ratio', 'lvpp')]
)


### 🔨 "Un-Pivoting"  Weighted Average Pivot to Prepare for Geometry 
🗂️ weighted_avg_pivot → weighted_average_metrics

In [ ]:
# Resetting index of weighted average pivot to create weighted_average_metrics DataFrame
weighted_average_metrics = weighted_avg_pivot.reset_index()

# Compressing hierarchy in columns and joining levels with "_"
weighted_average_metrics.columns = ['_'.join(str(col) for col in column) for column in weighted_average_metrics.columns.values]

# Removing trailing "_" introduced when compressing
for col in weighted_average_metrics.columns:
        # Check if the column name ends with '_'
        if col.endswith('_'):
            # If it does, remove the trailing underscore
            weighted_average_metrics = weighted_average_metrics.rename(columns={col: col[:-1]})

weighted_average_metrics.head()

In [ ]:
# List of unique system names in weighted assessments
weighted_average_metrics_system_names = weighted_average_metrics['system_name'].tolist()

# List of unique system names in tn_leas
tn_leas_system_names = tn_leas['system_name'].tolist()

# Checking if system names exist in weighted assessments and do not in the district boundaries data set
unique_in_weighted_average_metrics = list(set(weighted_average_metrics_system_names) - set(tn_leas_system_names))

# Check if system name exists in district boundaries, but does not in weighted assessments
unique_in_tn_leas = list(set(tn_leas_system_names) - set(weighted_average_metrics_system_names))

Checking if system names exist in weighted assessments that do not in the district boundaries data set.  I've never been so happy to see an empty list!

In [ ]:
unique_in_weighted_average_metrics

###  📊 🤝 🗺️ Merging Weighted Average Metrics with District Geometry Files and converting to GeoDataFrame

In [ ]:
weighted_average_metrics = pd.merge(left=weighted_average_metrics, right=tn_leas, on="system_name", how='left')
weighted_average_metrics = gpd.GeoDataFrame(weighted_average_metrics, geometry='geometry')

#### Renaming columns to conform with ERSI standards

In [ ]:

rename_dict = {
    'pct_met_exceeded_2018': 'pctm_18',
    'pct_met_exceeded_2019': 'pctm_19',
    'pct_met_exceeded_2021': 'pctm_21',
    'pct_met_exceeded_2022': 'pctm_22',
    'fte_teachers_2018': 'fte_18',
    'fte_teachers_2019': 'fte_19',
    'fte_teachers_2021': 'fte_21',
    'fte_teachers_2022': 'fte_22',
    'stu_tchr_ratio_2018': 'str_18',
    'stu_tchr_ratio_2019': 'str_19',
    'stu_tchr_ratio_2021': 'str_21',
    'stu_tchr_ratio_2022': 'str_22',
    'pct_met_exceeded_lvpp': 'pctm_lvpp',
    'pct_met_exceeded_intra': 'pctm_intra',
    'pct_met_exceeded_pre-post': 'pctm_pp',
    'fte_teachers_lvpp': 'fte_lvpp',
    'fte_teachers_intra': 'fte_intra',
    'fte_teachers_pre-post': 'fte_pp',
    'stu_tchr_ratio_lvpp': 'str_lvpp',
    'stu_tchr_ratio_intra': 'str_intra',
    'stu_tchr_ratio_pre-post': 'str_pp'
}
weighted_average_metrics.rename(columns=rename_dict, inplace=True)

In [ ]:
weighted_average_metrics.head(n=1)

There's some funk going on here.  District names don't match across datasets.  I'm going to do a spatial merge to see which dististricts are associated based on thier physical location.

#### 🥒 Exporting Weighted Average Metrics without Geospatial Info

In [ ]:
## CAUTION!: Don't turn on the pkl generator unless hungry.
weighted_average_metrics.to_pickle('../data/Weighted_average_metrics.pkl')  

###  🗺️ Exporting Weighed Average Metrics GeoDataFrame as Shapefile

In [ ]:
# CAUTION!: Don't turn on the GeoJSON generator unless lost.
# weighted_average_metrics.to_file('../data/weighted_average_metrics.geojson', driver='GeoJSON')

In [ ]:
# weighted_average_metrics.to_file('../data/weighted_average_metrics.shp', driver='ESRI Shapefile')

### 📊 Visual EDA

In [ ]:
# Specify the school level you want to focus on
selected_school_level = 'High'

# Filter the data for "All Students" student group and the selected school level
data = weighted_average_metrics[(weighted_average_metrics['student_group'] == 'All Students') & (weighted_average_metrics['school_lvl'] == selected_school_level)]

# Get the unique content areas for the selected school level
content_areas = data['subject_area'].unique()

# Create a 2D array to store the heatmap values
z_data = []

# Iterate over content areas
for content_area in content_areas:
    # Filter the data for the current content area
    content_area_data = data[data['subject_area'] == content_area]
    
    # Get the lagged calculation values for the current content area
    lagged_values = content_area_data['pctm_pp'].values
    
    # Append the lagged calculation values to the z_data list, ignoring NaN values
    z_data.append([value for value in lagged_values if not np.isnan(value)])

# Create the heatmap trace
heatmap = go.Heatmap(
    x=content_areas,
    y=['Lagged Calculation'],
    z=z_data,
    colorscale='viridis',
    colorbar=dict(title='Percentage')
)

# Create the figure and add the heatmap
fig = go.Figure(data=heatmap)

# Customize the layout
fig.update_layout(
    title=f'Lagged Calculation by Content Area for {selected_school_level} School Level',
    xaxis=dict(title='Content Area'),
    yaxis=dict(title=''),
    height=400,
    width=800
)

# Show the heatmap
fig.show()

In [ ]:
weighted_average_metrics.head()

In [ ]:
subject_area = [weighted_average_metrics['subject_area'].unique()]
subject_area

In [ ]:
pct_lags = weighted_average_metrics[['school_lvl', 'subject_area', 'pctm_lvpp', 'pctm_intra', 'pctm_pp']]
average_proficiency = pct_lags.groupby(['school_lvl', 'subject_area']).mean().reset_index()
average_proficiency

In [ ]:
# Calculate average proficiency per school level and content area
average_proficiency = weighted_average_metrics.groupby(['school_lvl', 'subject_area']).mean()[['pctm_lvpp', 'pctm_intra', 'pctm_pp']].reset_index()

# Define subject_area
subject_area = weighted_average_metrics['subject_area'].unique().tolist()

# Reorder school levels
school_lvl_order = ['Elementary', 'Middle', 'High', 'Secondary', 'Other']

# Create bar plot
fig = px.bar(average_proficiency, x='subject_area', y=['pctm_lvpp', 'pctm_intra', 'pctm_pp'], color_discrete_map={'pctm_lvpp': 'blue', 'pctm_intra': 'green', 'pctm_pp': 'red'},
             facet_row='subject_area', facet_col='school_lvl',
             category_orders={'subject_area': subject_area, 'school_lvl': school_lvl_order},
             labels={'pctm_lvpp': 'Average Proficiency (LVPP)',
                     'pctm_intra': 'Average Proficiency (Intra)',
                     'pctm_pp': 'Average Proficiency (PP)',
                     'school_lvl': 'School Level',
                     'subject_area': 'Subject Area'})

# Update layout
fig.update_layout(
    title='Average Weighted Proficiency by School Level and Content Area',
    autosize=True,
    width=1200,
    height=900,
)

# Show the figure
fig.show()

# Save the figure as HTML
fig.write_html('../plotly_html/broad_overview.html')